In [1]:
from asyncio import get_event_loop
from websockets import serve
from matplotlib import pyplot as plt
from matplotlib import patches
from numpy import array
from skimage import io, transform
from skimage import color, data, restoration
from collections import Counter
from pylibdmtx.pylibdmtx import decode
from multiprocessing import Pool
from io import BytesIO, StringIO
from json import dumps, loads, load
from uuid import uuid4
from math import cos, sin, pi
import base64
from glob import glob
%matplotlib inline

In [2]:
def decode_thread(pp_more):
    well = array(pp_more['well'])
    if well.shape[0] > 0 and well.shape[1] > 0:
        # try shrink
        res = decode(well, shrink=2, max_count=1)
        if res and res[0].data.decode().isnumeric():
            return {**pp_more['pp'], 'barcode': res[0].data.decode()}
        # try defaults
        res = decode(well, max_count=1)
        if res and res[0].data.decode().isnumeric():
            return {**pp_more['pp'], 'barcode': res[0].data.decode()}
        # try threshold
        res = decode(well, threshold=100, max_count=1)
        if res and res[0].data.decode().isnumeric():
            return {**pp_more['pp'], 'barcode': res[0].data.decode()}
    return {**pp_more['pp'], 'barcode': "failed"}


def map_wells(col, row, orientation):
    if orientation == 'landscape':  # "normal". A1 is top left
        return f"{['A','B','C','D','E','F','G','H'][row]}{col+1}"
    else:  # rotated 90 degrees right
        return f"{['H','G','F','E','D','C','B','A'][col]}{row+1}"


def analyze_image(image, pps):
    wells = [{'pp':pp, 'well': image[pp['minY']:pp['maxY'], pp['minX']:pp['maxX']]} for pp in pps]
    with Pool(8) as p:
        rar = p.map(decode_thread, wells)
    return rar

In [3]:
print(glob("../*.json"))
files = glob("../*.json")

['../32350698-0848-4a0e-a347-6266303819c0.json', '../331e62a9-b144-4842-aa07-c2f21ae02986.json', '../a7c8c6a9-485b-4cab-b5f5-f3af743b8a56.json', '../f530df66-311f-4c45-94cc-738caec8cbca.json', '../6a29c940-06a0-4de3-af9c-6820fe240999.json', '../c50bcc6c-6639-4601-be23-060c803b00ed.json', '../0f37293e-dd4b-4691-bbce-5778e997c0e7.json']


In [9]:
print(image_zero.shape)
print(len(images))
#(1440, 1920,


(1920, 1080, 4)
5


In [8]:
image_zero.shape

(1920, 1080, 4)

In [5]:
all_data = load(open(files[1], 'r'))

In [6]:
grid = all_data['grid']
iscale = all_data['scale']
images = all_data['images']
image_zero = io.imread(BytesIO(base64.decodebytes(images[0]['src'].split(',')[1].encode())))

In [7]:
scale = image_zero.shape[1] / iscale
if grid['width'] > grid['height']:
    grid_size = (grid['width'] / 12) * scale
    orientation = "landscape"
    no_rows = 8
    no_cols = 12
else:
    grid_size = (grid['height'] / 12) * scale
    orientation = "portrait"
    no_rows = 12
    no_cols = 8
width = grid_size * grid['scaleX']
height = grid_size * grid['scaleY']
ori_x = grid['left'] * scale
ori_y = grid['top'] * scale
angle = (grid['angle'] / 360) * 2 * pi
pps = []
for row in range(no_rows):
    for col in range(no_cols):
        dx1 = ori_x + width * col * cos(angle) - height * row * sin(angle)
        dx2 = ori_x + width * col * cos(angle) - height * (row + 1) * sin(angle)
        dx3 = ori_x + width * (col + 1) * cos(angle) - height * (row + 1) * sin(angle)
        dx4 = ori_x + width * (col + 1) * cos(angle) - height * row * sin(angle)
        dxs = [dx1, dx2, dx3, dx4]
        dy1 = ori_y + width * col * sin(angle) + height * row * cos(angle)
        dy2 = ori_y + width * col * sin(angle) + height * (row + 1) * cos(angle)
        dy3 = ori_y + width * (col + 1) * sin(angle) + height * (row + 1) * cos(angle)
        dy4 = ori_y + width * (col + 1) * sin(angle) + height * row * cos(angle)
        dys = [dy1, dy2, dy3, dy4]

        pps.append({'col':col, 'row': row, 'minY': int(min(dys)), 'maxY': int(max(dys)), 'minX': int(min(dxs)), 'maxX': int(max(dxs)), 'x0': dx1, 'y0':dy1})


results = [analyze_image(io.imread(BytesIO(base64.decodebytes(image['src'].split(',')[1].encode()))), pps) for image in images]

# analyze results
rar = [] #list of well objects with {'row', 'col', 'barcode', 'x0', 'y0'}
for well_no in range(no_rows*no_cols):
    #check that its the same well
    if len(set([results[i][well_no]['col'] for i in range(len(results))]))==1 and len(set([results[i][well_no]['row'] for i in range(len(results))]))==1:
        barcodes = [results[i][well_no]['barcode'] for i in range(len(results)) if results[i][well_no]['barcode'] != 'failed']
        barcodes_set = set(barcodes)
        if len(barcodes_set) == 1:
            rar.append({**results[0][well_no], 'barcode':barcodes_set.pop()})
        elif len(barcodes_set) == 0:
            rar.append({**results[0][well_no], 'barcode':'failed'})
        else:
            #lets see if theres a concensus
            barcodes_counter = Counter(barcodes)
            two_most_common = barcodes_counter.most_common(2)
            if two_most_common[0][1] == two_most_common[1][1]: # cant decide
                rar.append({**results[0][well_no], 'barcode': f"uncertain: {barcodes}"})
            else:
                rar.append({**results[0][well_no], 'barcode': two_most_common[0][0]})
    else:
        print("error...not same well?")

# draw result consensus
fig, ax = plt.subplots(1, figsize=(10, 20))
ax.imshow(image_zero)
rect = patches.Rectangle((grid['left'] * scale, grid['top'] * scale), width=grid['width'] * grid['scaleX'] * scale,
                                                 height=grid['height'] * grid['scaleY'] * scale, angle=grid['angle'], linewidth=1,
                                                 edgecolor='r', facecolor='none')
ar_pa = patches.Arrow(grid['left'] * scale, grid['top'] * scale, cos(((grid['angle'] - 90) / 360) * (2 * pi)) * 50,
                                            sin(((grid['angle'] - 90) / 360) * (2 * pi)) * 50, color='green', width=10)
ax.add_patch(rect)
ax.add_patch(ar_pa)

for irar in rar:
    if irar['barcode'] == 'failed':
        color = 'red'
        lw = 3
    elif 'uncertain' in irar['barcode']:
        color = 'blue'
        lw = 3
    else:
        color = 'green'
        lw = 1
    mrect = patches.Rectangle((irar['x0'], irar['y0']), width=width, height=height, angle=grid['angle'], linewidth=lw, edgecolor=color,facecolor='none')
    ax.add_patch(mrect)


plt.show()
#return
rrar = [{'loc': map_wells(entry['col'], entry['row'], orientation), 'barcode': entry['barcode']} for entry in rar]

ValueError: too many values to unpack (expected 3)

In [ ]:
wells = [{'pp':pp, 'well': image[pp['minY']:pp['maxY'], pp['minX']:pp['maxX']]} for pp in pps]
results = [analyze_image(io.imread(BytesIO(base64.decodebytes(image['src'].split(',')[1].encode()))), pps) for image in images]


In [15]:
billed0 = io.imread(BytesIO(base64.decodebytes(images[0]['src'].split(',')[1].encode())))
pp0 = pps[0]
brond = billed0[pp0['minY']:pp0['maxY'], pp0['minX']:pp0['maxX']]

In [29]:
decode(np.array(brond))

[Decoded(data=b'0170878908', rect=Rect(left=98, top=85, width=-35, height=-46))]

In [28]:
import numpy as np
type(np.array(brond))

numpy.ndarray

In [59]:
image, timeout=None, gap_size=None, shrink=1, shape=None,
           deviation=None, threshold=None, min_edge=None, max_edge=None,
           corrections=None, max_count=None):

dict